In [1]:
import mne
from anlffr.helper import biosemi2mne as bs
import sys
import warnings
from anlffr.preproc import find_blinks
from matplotlib import pyplot as plt
from mne import compute_proj_epochs
from mne import concatenate_epochs
import pylab as pl
import numpy as np
import os
import fnmatch
from mne.time_frequency import tfr_multitaper
from scipy.fft import fft
from scipy.signal import find_peaks
import alpha_spec_functions 
import gc 
import tracemalloc

In [2]:
fpath = '/Users/sharadhib/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Attachments/Attention_EEG/'
subjects = ['HL002', 'HL004', 'HL005', 'HL008', 'HL010', 'HL007','SP003', 'SP004', 'SP005', 'SP006', 'SP007', 'SP008' , 'HR']
# subjects = ['HR']
HL_subjects = ['HL002', 'HL004', 'HL005', 'HL008', 'HL010', 'HL007']
NH_subjects = ['SP003', 'SP004', 'SP005', 'SP006', 'SP007', 'SP008' , 'HR']

In [3]:
resultsdir = 'results/'

tracemalloc.start()

for subj in subjects:
    print(subj)
    lefts = []
    rights = []
    cues = []
    leads = []
    lags= []
    
    channels_to_drop = ['GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp']
    
    bdfs = fnmatch.filter(os.listdir(fpath), subj + '_Att*.bdf')
    if subj == 'HR':
        bdfs = ['HR_Attention+006.bdf', 'HR_Attention+007.bdf', 'HR_Attention+005.bdf', 'HR_Attention+004.bdf', 'HR_Attention+001.bdf', 'HR_Attention+003.bdf', 'HR_Attention+002.bdf']
    print(bdfs)
    
    for k, bdf in enumerate(bdfs):
        if bdf == 'HL008_AttentionTask+004.bdf':
            raw, eves = bs.importbdf(bdf, hptsname='biosemi10_20.hpts')
            total = raw.times[-1]
            end_time = total - 120
            raw = raw.crop(tmin=0, tmax=end_time)
        else:
            raw, eves = bs.importbdf(bdf, hptsname='biosemi10_20.hpts')
        print(f'Running file: {bdf}')
        raw.notch_filter(np.arange(60, 241, 60), filter_length='auto', fir_design='firwin')
        raw.filter(1, 40)

        if subj == 'SP004':
            raw.drop_channels(channels_to_drop)

        blinks = find_blinks(raw, ch_name=['Fp1'])
        if subj in ['HR', 'SP004', 'HL004', 'HL010', 'HL007']:
            if k == 0:
                epochs_blinks = mne.Epochs(raw, blinks, event_id=998, baseline=(-0.25, 0.25),
                                        reject=dict(eeg=500e-6), tmin=-0.25, tmax=0.25)
                blink_proj = compute_proj_epochs(epochs_blinks, n_eeg=2)
            
            raw.add_proj(blink_proj)
            raw.apply_proj()
        
        else:
            if k == 0:
                epochs_blinks = mne.Epochs(raw, blinks, event_id=998, baseline=(-0.25, 0.25),
                                        reject=dict(eeg=500e-6), tmin=-0.25, tmax=0.25)
                blink_proj = compute_proj_epochs(epochs_blinks, n_eeg=3)
            
            raw.add_proj(blink_proj)
            raw.del_proj(1)
            raw.apply_proj()
    
        events_lead = [7,9]
        events_lead_use = [event_lead for event_lead in events_lead if event_lead in eves[:,2]]
        
        events_lag = [8,10]
        events_lag_use = [event_lag for event_lag in events_lag if event_lag in eves[:,2]]

        events_left = [7,8]
        events_left_use = [event_left for event_left in events_left if event_left in eves[:,2]]
    
        events_right = [9,10]
        events_right_use = [event_right for event_right in events_right if event_right in eves[:,2]]
    
        if events_lead_use:
            epochs_lead = mne.Epochs(raw, eves, event_id=events_lead_use, baseline=(-0.5, 0.0), proj=True,
                            tmin=-0.5, tmax=5, reject=dict(eeg=200e-6))
            leads.append(epochs_lead)
            
        if events_lag_use:
            epochs_lag = mne.Epochs(raw, eves, event_id=events_lag_use, baseline=(-0.5, 0.0), proj=True,
                            tmin=-0.5, tmax=5, reject=dict(eeg=200e-6))
            lags.append(epochs_lag)
            
        if events_left_use:
            epochs_left = mne.Epochs(raw, eves, event_id=events_left_use, baseline=(-0.5, 0.0), proj=True,
                        tmin=-0.5, tmax=5, reject=dict(eeg=200e-6))
        lefts.append(epochs_left)
    
        if events_right_use:
            epochs_right = mne.Epochs(raw, eves, event_id=events_right_use, baseline=(-0.5, 0.0), proj=True,
                        tmin=-0.5, tmax=5, reject=dict(eeg=200e-6))
        rights.append(epochs_right)

        leads += [epochs_lead, ]
        lags += [epochs_lag, ]
    
    if leads:
        lead = concatenate_epochs(leads)
    if lags:
        lag = concatenate_epochs(lags) 
    if lefts:
        left = concatenate_epochs(lefts)
    if rights:
        right = concatenate_epochs(rights)

    del leads, lags, lefts, rights
    gc.collect()

    del raw, eves
    gc.collect()

    # epoch_left_pre = left.copy().crop(tmin=0.4, tmax=1.6)
    # epoch_right_pre = right.copy().crop(tmin=0.4, tmax=1.6) 

    epoch_left_dur = left.copy().crop(tmin=2.0, tmax=4.4)
    epoch_right_dur = right.copy().crop(tmin=2.0, tmax=4.4) 
    
    del left, right, lead, lag 

    freqs = np.arange(6., 14., 1.)
    n_cycles = freqs * 0.4

    # power_alpha_left_pre = epoch_left_pre.compute_tfr(method="multitaper", freqs=freqs, picks=['eeg'], return_itc=False, n_cycles=n_cycles)
    # power_alpha_right_pre = epoch_right_pre.compute_tfr(method="multitaper", freqs=freqs, picks=['eeg'], return_itc=False, n_cycles=n_cycles)

    # del epoch_left_pre, epoch_right_pre
    
    # fname_pleft_pre = fpath+resultsdir+subj+ '_left_alpha_prestim_new_tfr.h5'
    # fname_pright_pre = fpath+resultsdir+subj+ '_right_alpha_prestim_new_tfr.h5'
    
    # power_alpha_left_pre.save(fname_pleft_pre, overwrite=True)
    # power_alpha_right_pre.save(fname_pright_pre, overwrite=True)

    power_alpha_left_dur = epoch_left_dur.compute_tfr(method="multitaper", freqs=freqs, picks=['eeg'], return_itc=False, n_cycles=n_cycles)
    power_alpha_right_dur = epoch_right_dur.compute_tfr(method="multitaper", freqs=freqs, picks=['eeg'], return_itc=False, n_cycles=n_cycles)

    del epoch_left_dur,epoch_right_dur

    fname_pleft_dur = fpath+resultsdir+subj+ '_left_alpha_dur_new_tfr.h5'
    fname_pright_dur = fpath+resultsdir+subj+ '_right_alpha_dur_new_tfr.h5'

    power_alpha_left_dur.save(fname_pleft_dur, overwrite=True)
    power_alpha_right_dur.save(fname_pright_dur, overwrite=True)


    # avg_power_alpha_left_pre = power_alpha_left_pre.data.mean(axis=0).mean(axis=1).mean(axis=1)
    # avg_power_alpha_right_pre = power_alpha_right_pre.data.mean(axis=0).mean(axis=1).mean(axis=1)

    avg_power_alpha_left_dur = power_alpha_left_dur.data.mean(axis=0).mean(axis=1).mean(axis=1)
    avg_power_alpha_right_dur = power_alpha_right_dur.data.mean(axis=0).mean(axis=1).mean(axis=1)
    
    # del power_alpha_left_dur, power_alpha_right_dur, power_alpha_left_pre, power_alpha_right_pre

    # norm_avg_power_alpha_right_left_pre = (avg_power_alpha_right_pre - avg_power_alpha_left_pre)/((avg_power_alpha_left_pre + avg_power_alpha_right_pre)*0.5)*100 

    norm_avg_power_alpha_right_left_dur = (avg_power_alpha_right_dur - avg_power_alpha_left_dur)/(avg_power_alpha_left_dur + avg_power_alpha_right_dur)*0.5*100

    # fname_left_right_pre = fpath+resultsdir+subj+ '_norm_right_left_preprestim'
    # np.save(fname_left_right_pre, norm_avg_power_alpha_right_left_pre)

    fname_left_right_dur = fpath+resultsdir+subj+ '_norm_right_left_dur'
    np.save(fname_left_right_dur, norm_avg_power_alpha_right_left_dur)

    # del avg_power_alpha_left_pre, avg_power_alpha_right_pre, norm_avg_power_alpha_right_left_pre, 
    del avg_power_alpha_left_dur,avg_power_alpha_right_dur, norm_avg_power_alpha_right_left_dur
    gc.collect()

tracemalloc.stop()

HL002
['HL002_AttentionTask+001.bdf', 'HL002_AttentionTask+003.bdf', 'HL002_AttentionTask+002.bdf', 'HL002_AttentionTask+004.bdf', 'HL002_AttentionTask.bdf']
Extracting EDF parameters from /Users/sharadhib/Library/CloudStorage/OneDrive-UniversityofPittsburgh/Attachments/Attention_EEG/HL002_AttentionTask+001.bdf...
BDF file detected


Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 8716287  =      0.000 ...   532.000 secs...
Re-referencing data to: EXG1 EXG2EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Trigger channel Status has a non-zero initial value of {initial_value} (consider using initial_event=True to detect this event)
119 events found on stim channel Status
Event IDs: [ 4  5  7  8  9 10 11]
Running file: HL002_AttentionTask+001.bdf
Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 108135 samples (6.600 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.0s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 54069 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.2s


...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    3.1s


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 54069 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.4s


Setting up band-pass filter from 0.5 - 10 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.15 Hz (-6 dB cutoff frequency: 0.42 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 2.50 Hz (-6 dB cutoff frequency: 11.25 Hz)
- Filter length: 360449 samples (22.000 s)

2 projection items deactivated
Created an SSP operator (subspace dimension = 2)
2 projection items activated
SSP projectors applied...
Not setting metadata
18 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Not setting metadata
15 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 2)
2 projection items activated
Not settin

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   24.8s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:   25.4s
